In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import joblib

# Define the path to the dump files folder
dump_folder = r"C:\Users\MAY02\Desktop\SLM_NeuralNetwork\Relax_Al_helix_1_1"

# Function to process multiple dump files
def process_dump_files(dump_files):
    all_data = []
    for file in dump_files:
        with open(file, 'r') as f:
            lines = f.readlines()
        
        # Find the relevant section with atom data
        start_reading = False
        atom_data = []
        for line in lines:
            if "ITEM: ATOMS" in line:
                start_reading = True
                headers = line.strip().split()[2:]
                continue
            if start_reading:
                atom_data.append(line.strip().split())
        
        # Convert to DataFrame
        df = pd.DataFrame(atom_data, columns=headers)
        df = df.astype(float)
        
        # Add timestep column (extracted from filename or content)
        timestep = int(file.split("coord")[-1].split(".dump")[0])
        df["timestep"] = timestep
        
        all_data.append(df)
    
    return pd.concat(all_data, ignore_index=True)

# Get all dump files
dump_files = glob.glob(os.path.join(dump_folder, "coord*.dump"))
print(f"Found {len(dump_files)} dump files in {dump_folder}.")

# Process all dump files
df_atoms = process_dump_files(dump_files)

# Load external simulation parameters
data_params = pd.read_csv(os.path.join(dump_folder, "slm_heat_transfer_data.csv"))

# Merge with extracted atom data
data = pd.merge(df_atoms, data_params, how="cross")

# Define input and output columns
input_features = ["laser_power", "scan_speed", "layer_thickness", "powder_density", "x", "y", "z", "timestep"]
target_feature = "f_Temp[0]"

# Extract inputs and outputs
X = data[input_features].values
y = data[target_feature].values.reshape(-1, 1)

# Normalize data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train LightGBM Regressor
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_test = lgb.Dataset(X_test, label=y_test, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.1,
    'num_leaves': 31
}

print("Training LightGBM ...")
model = lgb.train(
    params,
    lgb_train,
    num_boost_round=100,
    valid_sets=[lgb_train, lgb_test],
    valid_names=["train","valid"],
)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Save model
joblib.dump(model, os.path.join(dump_folder, "heat_transfer_lgbm_model.pkl"))
print("LightGBM Model trained on multiple timesteps and saved in the specified folder.")


Found 241 dump files in C:\Users\MAY02\Desktop\SLM_NeuralNetwork\Relax_Al_helix_1_1.


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\MAY02\\Desktop\\SLM_NeuralNetwork\\Relax_Al_helix_1_1\\slm_heat_transfer_data.csv'